In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
from collections import Counter

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df_train = pd.read_csv('train.csv')
df_test  = pd.read_csv('test.csv')
df_sample= pd.read_csv('titanic.csv')

In [3]:
### Define function to remove Outleirs data
def detect_outliers(df,n,features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers

In [4]:
Outliers_to_drop = detect_outliers(df_train,2,["Age","SibSp","Parch","Fare"])
df_train = df_train.drop(Outliers_to_drop, axis = 0).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:3652: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


In [5]:
def impute_fare(cols):
    Fare = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Fare):

        if Pclass == 1:
            return 84

        elif Pclass == 2:
            return 20

        else:
            return 13

    else:
        return Fare

In [7]:
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):

        if Pclass == 1:
            return 37

        elif Pclass == 2:
            return 29

        else:
            return 24

    else:
        return Age

In [8]:
df_train['Age'] = df_train[['Age','Pclass']].apply(impute_age,axis=1)

In [9]:
df_train['Age'] = df_train[['Age','Pclass']].apply(impute_age,axis=1)
sex = pd.get_dummies(df_train['Sex'],drop_first=True)
embark = pd.get_dummies(df_train['Embarked'],drop_first=True)
df_train = pd.concat([df_train,sex,embark],axis=1)
df_train["Family"] = df_train["SibSp"] + df_train["Parch"] + 1
df_train['Single'] = df_train['Family'].map(lambda s: 1 if s == 1 else 0)
df_train['SmallF'] = df_train['Family'].map(lambda s: 1 if  s == 2  else 0)
df_train['MedF']   = df_train['Family'].map(lambda s: 1 if 3 <= s <= 4 else 0)
df_train['LargeF'] = df_train['Family'].map(lambda s: 1 if s >= 5 else 0)
df_train['Senior'] = df_train['Age'].map(lambda s:1 if s>60 else 0)
dataset_title = [i.split(",")[1].split(".")[0].strip() for i in df_test["Name"]]
df_test["Title"] = pd.Series(dataset_title)
df_test["Title"] = df_test["Title"].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df_test["Title"] = df_test["Title"].map({"Master":0, "Miss":1, "Ms" : 1 , "Mme":1, "Mlle":1, "Mrs":1, "Mr":2, "Rare":3})
df_test["Title"] = df_test["Title"].astype(int)
df_test.drop(labels = ["Name"], axis = 1, inplace = True)
df_test['Age'] = df_test[['Age','Pclass']].apply(impute_age,axis=1)
sex = pd.get_dummies(df_test['Sex'],drop_first=True)
embark = pd.get_dummies(df_test['Embarked'],drop_first=True)
df_test = pd.concat([df_test,sex,embark],axis=1)
df_test['Fare'].fillna(value=df_test['Fare'].median(),inplace=True)
df_test['Fare'] = df_test[['Fare','Pclass']].apply(impute_fare,axis=1)
df_test["Fare"] = df_test["Fare"].map(lambda i: np.log(i) if i > 0 else 0)
df_test["Family"] = df_test["SibSp"] + df_test["Parch"] + 1
df_test['Single'] = df_test['Family'].map(lambda s: 1 if s == 1 else 0)
df_test['SmallF'] = df_test['Family'].map(lambda s: 1 if  s == 2  else 0)
df_test['MedF']   = df_test['Family'].map(lambda s: 1 if 3 <= s <= 4 else 0)
df_test['LargeF'] = df_test['Family'].map(lambda s: 1 if s >= 5 else 0)
df_test['Senior'] = df_test['Age'].map(lambda s:1 if s>60 else 0)

In [10]:
def get_person(passenger):
    age,sex = passenger
    return 'child' if age < 16 else sex

In [11]:
df_train['Person'] = df_train[['Age','Sex']].apply(get_person,axis=1)
df_test['Person']  = df_test[['Age','Sex']].apply(get_person,axis=1)

person_dummies_train  = pd.get_dummies(df_train['Person'])
person_dummies_train.columns = ['Child','Female','Male']
person_dummies_train.drop(['Male'], axis=1, inplace=True)

person_dummies_test  = pd.get_dummies(df_test['Person'])
person_dummies_test.columns = ['Child','Female','Male']
person_dummies_test.drop(['Male'], axis=1, inplace=True)

df_train = df_train.join(person_dummies_train)
df_test  = df_test.join(person_dummies_test)

df_train.drop(['Person'],axis=1,inplace=True)
df_test.drop(['Person'],axis=1,inplace=True)
df_train.drop('male',axis=1,inplace=True)
df_test.drop('male',axis=1,inplace=True)
df_train.drop(['Cabin','Ticket'],axis = 1, inplace= True)
df_test.drop(['Ticket','Cabin'],axis = 1, inplace= True)
df_train.drop(['Sex','Embarked','Name'],axis=1,inplace=True)
df_test.drop(['Sex','Embarked','Name'],axis=1,inplace=True)

ValueError: labels ['Name'] not contained in axis

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train.drop('Survived',axis=1), 
                                                    df_train['Survived'], test_size=0.20, 
                                                    random_state=101)

In [96]:
X_train.shape
X_test.shape

(177, 16)

In [68]:
y_train = pd.DataFrame(y_train)


In [69]:
y_train.shape

(704, 1)

In [80]:
from keras import models
from keras import layers
from keras import optimizers

In [81]:
model = models.Sequential()

In [82]:
model.add(layers.Dense(512,activation='relu',input_shape=(16,)))
model.add(layers.Dense(10,activation='softmax'))

In [83]:
model.compile(optimizer=optimizers.RMSprop(lr=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [86]:
model.fit(X_train,y_train,batch_size=2,epochs=10)

Epoch 1/10
704/704 [==============================] - 0s 387us/step - loss: 0.6420 - acc: 0.8054
Epoch 2/10
704/704 [==============================] - 0s 385us/step - loss: 0.6124 - acc: 0.7983
Epoch 3/10
704/704 [==============================] - 0s 384us/step - loss: 0.5651 - acc: 0.8196
Epoch 4/10
704/704 [==============================] - 0s 370us/step - loss: 0.5834 - acc: 0.7955
Epoch 5/10
704/704 [==============================] - 0s 396us/step - loss: 0.5818 - acc: 0.8011
Epoch 6/10
704/704 [==============================] - 0s 361us/step - loss: 0.5587 - acc: 0.7983
Epoch 7/10
704/704 [==============================] - 0s 372us/step - loss: 0.5570 - acc: 0.8097
Epoch 8/10
704/704 [==============================] - 0s 382us/step - loss: 0.5650 - acc: 0.7955
Epoch 9/10
704/704 [==============================] - 0s 370us/step - loss: 0.5308 - acc: 0.8068
Epoch 10/10
704/704 [==============================] - 0s 375us/step - loss: 0.5518 - acc: 0.8168


In [87]:
y_test = pd.DataFrame(y_test)
test_loss,test_acc = model.evaluate(X_test,y_test)

177/177 [==============================] - 0s 420us/step


In [88]:
test_acc

0.8079096055300222

In [89]:
len(y_test)

177

In [99]:
df_test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Q,S,Family,Single,SmallF,MedF,LargeF,Senior,Child,Female
0,892,3,male,34.5,0,0,2.057860,Q,2,1,0,1,1,0,0,0,0,0,0
1,893,3,female,47.0,1,0,1.945910,S,1,0,1,2,0,1,0,0,0,0,1
2,894,2,male,62.0,0,0,2.270836,Q,2,1,0,1,1,0,0,0,1,0,0
3,895,3,male,27.0,0,0,2.159003,S,2,0,1,1,1,0,0,0,0,0,0
4,896,3,female,22.0,1,1,2.508582,S,1,0,1,3,0,0,1,0,0,0,1
5,897,3,male,14.0,0,0,2.221917,S,2,0,1,1,1,0,0,0,0,1,0
6,898,3,female,30.0,0,0,2.031983,Q,1,1,0,1,1,0,0,0,0,0,1
7,899,2,male,26.0,1,1,3.367296,S,2,0,1,3,0,0,1,0,0,0,0
8,900,3,female,18.0,0,0,1.978128,C,1,0,0,1,1,0,0,0,0,0,1
9,901,3,male,21.0,2,0,3.184284,S,2,0,1,3,0,0,1,0,0,0,0


ValueError: Error when checking input: expected dense_19_input to have shape (16,) but got array with shape (1,)